# Get data and packages

In [1]:
%%capture
!pip install -U pip
!conda update -y --all
!pip install --upgrade tensorflow==2.0.0-beta1
!pip install --upgrade turicreate
!aws s3 cp --recursive s3://xdss-public-datasets/demos/taxi_1m.sframe ../datasets/taxi_1m.sframe

## Prep benchmarks

In [21]:
import gc
from src.benchmarks_utils import benchmark, get_results
from src.turicreate_utils import *

name = 'turicreate'
data_path = '../datasets/taxi_1m.sframe'
results_path = f"../results/{name}_1m.csv"
repetitions = 1
benchmarks = {}

# Benchmarks

In [22]:
# Load data
data = read_file(data_path=data_path)
data['pickup_hour'] = data['pickup_datetime'].str_to_datetime().apply(lambda x: x.hour)
print(f"size: {data.shape[0]} with {data.shape[1]} columns")

size: 1000000 with 19 columns


In [ ]:
benchmarks['read_file']= benchmark(read_file, df=data, data_path=data_path, repetitions=repetitions)
benchmarks['mean']= benchmark(mean, data, repetitions=repetitions)
benchmarks['standard deviation']= benchmark(standard_deviation, data, repetitions=repetitions)
benchmarks['sum columns']= benchmark(sum_columns, data, repetitions=repetitions)
benchmarks['product columns']= benchmark(product_columns, data, repetitions=repetitions)
benchmarks['arithmetic operation']= benchmark(complicated_arithmetic_operation, data, repetitions=repetitions)
benchmarks['value counts']= benchmark(value_counts, data, repetitions=repetitions)
benchmarks['groupby statistics']= benchmark(groupby_statistics, data, repetitions=repetitions)
benchmarks['filter']= benchmark(filter_data, data, repetitions=repetitions)
print(f"cleaned {gc.collect()} mb")
benchmarks['join'] = benchmark(join, data, repetitions=repetitions, other=groupby_statistics(data))
gc.collect()

# Sketch
benchmarks['sketch mean'] = benchmark(skeatch_mean, data, repetitions=repetitions)
gc.collect()
benchmarks['sketch standard deviation'] = benchmark(skeatch_standatd_deviation, data, repetitions=repetitions)
gc.collect()
benchmarks['skeatch frequent items'] = benchmark(skeatch_frequent_items, data, repetitions=repetitions)
print(f"Done benchmarks on all data")

# filtered
filterd = filter_data(data)
del data

print(f"Prepare filtered data and deleted {gc.collect()} MB")
benchmarks['filtered mean'] = benchmark(mean, filterd, repetitions=repetitions)
benchmarks['filtered standard deviation'] = benchmark(standard_deviation, filterd, repetitions=repetitions)
benchmarks['filtered sum columns'] = benchmark(sum_columns , filterd, repetitions=repetitions)
benchmarks['filtered product_columns'] = benchmark(product_columns , filterd, repetitions=repetitions)
benchmarks['filtered complicated arithmetic operation'] = benchmark(complicated_arithmetic_operation, filterd, repetitions=repetitions)
benchmarks['filtered value counts'] = benchmark(value_counts, filterd, repetitions=repetitions)
benchmarks['filtered groupby statistics'] = benchmark(groupby_statistics, filterd, repetitions=repetitions)
benchmarks['filtered join'] = benchmark(join, filterd, repetitions=repetitions, other=groupby_statistics(filterd))

# Sketch
benchmarks['sketch mean'] = benchmark(skeatch_mean, filterd, repetitions=repetitions)
gc.collect()
benchmarks['sketch standard deviation'] = benchmark(skeatch_standatd_deviation, filterd, repetitions=repetitions)
gc.collect()
benchmarks['skeatch frequent items'] = benchmark(skeatch_frequent_items, filterd, repetitions=repetitions)
print(f"Done benchmarks on filterd data")

In [ ]:
results = get_results(benchmarks, name)
results.to_csv(results_path)
results.head()

In [ ]:
!aws s3 cp  ../results/turicreate_1m.csv s3://vaex-sagemaker-demo/benchmarks/turicreate_1m_results.csv 